In [11]:
import numpy as np
import pandas as pd
import os

### Download videos for training

Downloads videos as mp4 and saves the audio as mp3. mp4 may or may not contain audio due to the API being limited, though this should not matter (except for the small negative impact on file size). Currently videos are being downloaded in 720p since 1080p seems bugged

In [12]:
from pytube import YouTube 
  
urls = ["https://www.youtube.com/watch?v=Yt3-a9mExZg", "https://www.youtube.com/watch?v=cGt8bEcd9Ms"]
pathVideo = 'Data/Video'
pathAudio = 'Data/Audio'

for i, url in enumerate(urls):
    yt = YouTube(url)
    yt.streams.filter(abr="160kbps", progressive=False).first().download(filename="Audio"+str(i)+".mp3", output_path = pathAudio)
    yt.streams.filter(res="720p", progressive=False).first().download(filename="Video"+str(i)+".mp4", output_path= pathVideo)
##

### Split video into frames

Goes through each downloaded video. Creates a directory per video inside 'tempFrames' and then saves the frames here. We can set the amount of frames we want per minute. Note that file sizes of these images can balloon quickly, exceeding the filesize of the video itself.

In [13]:
import cv2
FRAMES_PER_MINUTE = 15
frame_counter = 60000 / FRAMES_PER_MINUTE
pathVideo = 'Data/Video/'
videoNames = []
for filename in os.listdir(pathVideo):
    f = os.path.join(pathVideo, filename)
    if os.path.isfile(f): ## checking if it is a file
        videoNames.append(filename)

for video in videoNames:
    path = pathVideo + video
    outputPath = 'Data/tempFrames/' + video[:-4] + '/' ## make directory per video
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
        print(f'Created new dir {outputPath}')
    vidcap = cv2.VideoCapture(pathVideo + video)
    success,image = vidcap.read()
    print(f'Succesful videocapture?: {success}')
    count = 0
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_counter))
        cv2.imwrite(outputPath + "frame%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        count += 1

Created new dir Data/tempFrames/Video0/
Succesful videocapture?: True
Created new dir Data/tempFrames/Video1/
Succesful videocapture?: True


### Segmenting faces/webcam feeds

By segmenting individuals, we can create temporary folders containing the frames per individual participant. This might be neccessary depending on the emotion detection model that we apply. It seems better to avoid this step is possible, as it introduces an extra model (segmentation), and also requires the creation of more files which need to be operated on. This could be rather inefficient

In [14]:
## Some code here for extracting individuals, saving new cropped images

### Video emotion detection

Now we run an emotion detection model to acquire some score. For every frame, we should get some "emotion scores" per individual. We can then extract some values such as minimum, maximum and mean scores per person. We can eventually combine this into a final score. For this to be as accurate as possible we will measure audio later 

In [15]:
## Some code here

### Audio analysis